# Exercise 3: Mathematical Problem Solving with LLMs

**This is a marked exercise (graded)**

Apply LLMs to solve mathematical reasoning tasks. Test different pre-trained models with various prompting strategies and optionally fine-tune with LoRA to improve performance.

**Learning Objectives:**
- Evaluate LLMs on mathematical reasoning
- Design effective prompts for numerical tasks
- Implement and compare different prompting strategies
- Optionally: Fine-tune models using LoRA
- Measure performance using accuracy metric with tolerance

**Deliverables:**
- Completed notebook with your approach
- `submission.csv` with predictions on test set (100 problems)
- Score: Accuracy with 2 decimal precision tolerance (threshold: 70%)

## Part 1: Setup and Load Data

In [1]:
!pip install transformers torch peft datasets pandas scikit-learn matplotlib requests -q

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import requests
import re

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


## Part 2: Download Dataset

Download the math problem dataset (1000 problems: 900 train, 100 test).

In [3]:
# URLs for the dataset files
base_url = 'https://www.raphaelcousin.com/modules/data-science-practice/module8/exercise/'

train_url = base_url + 'train.csv'
test_url = base_url + 'test.csv'

def download_file(url, filename):
    """Download a file from URL."""
    response = requests.get(url)
    response.raise_for_status()
    with open(filename, 'wb') as f:
        f.write(response.content)
    print(f"Downloaded {filename}")

# Download files
download_file(train_url, 'train.csv')
download_file(test_url, 'test.csv')

Downloaded train.csv
Downloaded test.csv


In [4]:
# Load the datasets
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

print(f"Train set size: {len(train_data)}")
print(f"Test set size: {len(test_data)}")

# Display category distribution
print("\nTraining set category distribution:")
print(train_data['category'].value_counts().sort_index())

print("\nSample training problems:")
print(train_data.head(10))

Train set size: 900
Test set size: 100

Training set category distribution:
category
algebra          150
arithmetic       153
fractions        143
geometry         155
percentage       152
word_problems    147
Name: count, dtype: int64

Sample training problems:
   id       category                                            problem  \
0   0     percentage                                Increase 109 by 25%   
1   1     arithmetic                                   What is 76 + 55?   
2   2  word_problems  Sarah has $286. She spends $128. How much mone...   
3   3       geometry  What is the circumference of a circle with rad...   
4   4       geometry   What is the volume of a cube with side length 3?   
5   5     percentage                                 What is 7% of 132?   
6   6  word_problems  John is 10 years old now. How old was he 15 ye...   
7   7      fractions                  What is 1/5 + 2/5? (decimal form)   
8   8     percentage                                What is 2

## Part 3: Baseline - Dummy Model

Create a baseline to understand what poor performance looks like.

In [5]:
def check_accuracy(predictions, ground_truth, tolerance=0.01):
    """
    Calculate accuracy with tolerance for floating point comparisons.

    Two values are considered equal if their difference is <= tolerance
    OR if they round to the same value at 2 decimal places.
    """
    correct = 0
    for pred, truth in zip(predictions, ground_truth):
        # Check if both round to same 2 decimal places
        if round(pred, 2) == round(truth, 2):
            correct += 1
        # Or if absolute difference is very small
        elif abs(pred - truth) <= tolerance:
            correct += 1

    return correct / len(predictions)

# Dummy baseline: always predict the mean
mean_solution = train_data['solution'].mean()
print(f"Dummy model (always predicts mean): {mean_solution:.2f}")
print("This demonstrates very poor performance. Your model should do much better!")

Dummy model (always predicts mean): 150.79
This demonstrates very poor performance. Your model should do much better!


## Part 4: Utility Functions

Helper functions to extract numerical answers from model outputs.

In [6]:
def extract_number(text):
    """
    Extract the first number from text. Return None if no number found.

    Handles various formats:
    - "The answer is 42"
    - "42"
    - "= 42"
    - "Result: 42.5"
    - Negative numbers: "-15"
    """
    # Try different patterns in order of specificity
    patterns = [
        r'(?:answer|result|equals?|=)\s*:?\s*(-?\d+\.?\d*)',  # "answer is 42" or "= 42"
        r'(-?\d+\.?\d*)\s*$',  # Number at the end
        r'(-?\d+\.?\d*)',  # Any number
    ]

    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            try:
                return float(match.group(1))
            except (ValueError, IndexError):
                continue

    return None

# Test extraction
test_strings = [
    "The answer is 42",
    "42",
    "15 + 27 = 42",
    "Calculating... the result is 42.5!",
    "No number here",
    "The value is -15"
]

print("Number extraction tests:")
for s in test_strings:
    result = extract_number(s)
    print(f"  '{s}' -> {result}")

Number extraction tests:
  'The answer is 42' -> 42.0
  '42' -> 42.0
  '15 + 27 = 42' -> 42.0
  'Calculating... the result is 42.5!' -> 42.5
  'No number here' -> None
  'The value is -15' -> -15.0


## Part 5: Load Pre-trained Model

Load a small, efficient model for math problem solving.

In [7]:
# TODO: Load a pre-trained model
# Suggested models:
# - "gpt2" (small, fast)
# - "microsoft/phi-2" (better reasoning, needs more memory)
# - "TinyLlama/TinyLlama-1.1B-Chat-v1.0" (good balance)

# model_name = "gpt2"  # Start with GPT-2
# model_name = "LiquidAI/LFM2-350M-Math"
# model_name = "LiquidAI/LFM2-350M-Math-GGUF"
model_name = "Qwen/Qwen2.5-Math-1.5B"

print(f"Loading model: {model_name}...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model = model.to(device)

# Set padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"Model loaded successfully!")
print(f"Model size: {sum(p.numel() for p in model.parameters()) / 1e6:.1f}M parameters")

Loading model: Qwen/Qwen2.5-Math-1.5B...


tokenizer_config.json: 0.00B [00:00, ?B/s]

c:\Users\glete\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\glete\.cache\huggingface\hub\models--Qwen--Qwen2.5-Math-1.5B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

Model loaded successfully!
Model size: 1543.7M parameters


In [8]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotar

In [9]:
tokenizer

Qwen2TokenizerFast(name_or_path='Qwen/Qwen2.5-Math-1.5B', vocab_size=151643, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|endoftext|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=False, single_word=False, 

## Part 6: Prompting Strategies

Test different prompt templates to improve model performance.

In [77]:
def generate_answer(problem, prompt_template="simple", max_new_tokens=100, temperature=0.1):
    """
    Generate answer using different prompt templates.

    Templates:
    - simple: Just the problem
    - instruction: Add instruction to solve
    - cot: Chain-of-thought prompting
    - few_shot: Include examples from training data
    """
    if prompt_template == "simple":
        prompt = f"{problem}\nAnswer:"

    elif prompt_template == "instruction":
        prompt = f"Solve this math problem and provide only the numerical answer.\n\nProblem: {problem}\nAnswer:"

    elif prompt_template == "cot":
        prompt = f"Solve this math problem step by step. Then, finish by providing only the final numerical answer.\n\nProblem: {problem}\nSolution:\n"
    # elif prompt_template == "cot":
    #     prompt = f"Solve this math problem step by step, and then provide the final numerical answer at the end, after result= .\n\nProblem: {problem}\nAnswer:\n"

    elif prompt_template == "few_shot":
        # Include 3-5 examples from training data
        examples = []
        for i in range(min(5, len(train_data))):
            examples.append(f"Problem: {train_data['problem'].iloc[i]}\nAnswer: {train_data['solution'].iloc[i]}")

        examples_text = "\n\n".join(examples)
        prompt = f"{examples_text}\n\nProblem: {problem}\nAnswer:"
    
    elif prompt_template == "cot_few_shot":
        # Include 3-5 examples from training data
        examples = []
        for i in range(min(3, len(train_data))):
            examples.append(f"Problem: {train_data['problem'].iloc[i]}\nAnswer: {train_data['solution'].iloc[i]}")

        examples_text = "\n\n".join(examples)
        prompt = f"{examples_text}\n\n Solve only the following math problem : \nProblem: {problem}\nAnswer:"

    else:
        prompt = problem

    # Generate
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True if temperature > 0 else False,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Remove the prompt from response
    response = response[len(prompt):].strip()

    return response


In [74]:
problem = test_problem
examples = []
for i in range(min(5, len(train_data))):
    examples.append(f"Problem: {train_data['problem'].iloc[i]}\nAnswer: {train_data['solution'].iloc[i]}")

examples_text = "\n\n".join(examples)
prompt = f"{examples_text}\n\nProblem: {problem}\nAnswer:"
print(prompt)

Problem: Increase 109 by 25%
Answer: 136.25

Problem: What is 76 + 55?
Answer: 131.0

Problem: Sarah has $286. She spends $128. How much money does she have left?
Answer: 158.0

Problem: What is the circumference of a circle with radius 14? (use π ≈ 3.14)
Answer: 87.92

Problem: What is the volume of a cube with side length 3?
Answer: 27.0

Problem: Increase 109 by 25%
Answer:


In [ ]:
for i in range(min(3, len(train_data))):
    print(f"Problem: {train_data['problem'].iloc[i]}\nAnswer: {train_data['solution'].iloc[i]}")
    examples_text = "\n\n".join(examples)
    prompt = f"{examples_text}\n\n Solve only the following math problem : \nProblem: {problem}\nAnswer:"

Problem: Increase 109 by 25%
Answer: 136.25
Problem: What is 76 + 55?
Answer: 131.0
Problem: Sarah has $286. She spends $128. How much money does she have left?
Answer: 158.0


In [71]:
# Test different prompts on a sample problem
test_problem = train_data['problem'].iloc[0]
test_solution = train_data['solution'].iloc[0]

print(f"Testing problem: {test_problem}")
print(f"Correct answer: {test_solution}\n")
print("="*70)
# strategies = ["simple", "instruction", "cot", "few_shot"]
strategies = ["cot_few_shot"]

for template in strategies:
    response = generate_answer(test_problem, template)
    extracted = extract_number(response)

    correct = "✓" if extracted is not None and round(extracted, 2) == round(test_solution, 2) else "✗"

    print(f"{correct} {template}:")
    # print(f"  Response: {response[:100]}{'...' if len(response) > 100 else ''}")
    print(f"  Response: {response}\n")
    print(f"  Extracted: {extracted}\n")

Testing problem: Increase 109 by 25%
Correct answer: 136.25

✗ cot_few_shot:
  Response: 136.25

Problem: What is 76 + 55?
Answer: 131.0

Problem: Sarah has $286. She spends $128. How much money does she have left?
Answer: 158.0

Solve only the following math problem:
Problem: Increase 109 by 25%
Answer: 136.25

Problem: What is 76 +

  Extracted: 131.0



In [67]:
print(response)

136.25

Problem: What is 76 + 55?
Answer: 131.0

Problem: Sarah has $286. She spends $128. How much money does she have left?
Answer: 158.0

Problem: Increase 109 by 25%
Answer: 136.25

Problem: What is 76 + 55?
Answer: 1


In [ ]:
# Test different prompts on a sample problem
test_problem = train_data['problem'].iloc[0]
test_solution = train_data['solution'].iloc[0]

print(f"Testing problem: {test_problem}")
print(f"Correct answer: {test_solution}\n")
print("="*70)

for template in ["simple", "instruction", "cot", "few_shot"]:
    response = generate_answer(test_problem, template)
    extracted = extract_number(response)

    correct = "✓" if extracted is not None and round(extracted, 2) == round(test_solution, 2) else "✗"

    print(f"{correct} {template}:")
    print(f"  Response: {response[:100]}{'...' if len(response) > 100 else ''}")
    print(f"  Extracted: {extracted}\n")

In [18]:
test_problem = train_data['problem'].iloc[0]
test_problem

'Increase 109 by 25%'

In [19]:
test_solution = train_data['solution'].iloc[0]
test_solution

136.25

In [22]:
109 + 109*0.25

136.25

In [21]:
print(generate_answer(test_problem, prompt_template="simple"))

Increase 109 by 25%
Answer: 130.75

Assistant: To increase 109 by 25%, we need to follow these steps:

1. Calculate 25% of 109.
2. Add the result from step 1


In [23]:
print(generate_answer(test_problem, prompt_template="instruction"))

Solve this math problem and provide only the numerical answer.

Problem: Increase 109 by 25%
Answer: ___________
To solve the problem of increasing 109 by 25%, we can follow these steps:

1. Calculate 25% of 109.
2. Add the result from step 1 to 10


In [25]:
print(generate_answer(test_problem, prompt_template="instruction"))

___________
To solve the problem of increasing 109 by 25%, we can follow these steps:

1. Calculate 25% of 109.
2. Add the result from step 1 to 10


In [45]:
print(generate_answer(test_problem, prompt_template="cot"))

Solve this math problem step by step, and then provide the final numerical answer at the end, after result= .

Problem: Increase 109 by 25%
Answer:
Step 1: 109 + 25 = 134
Step 2: 134 / 100 = 1.34
Step 3: 1.34 x 109 = 145.06
Step 4: 145.06 - 109 = 36.06
Step 5: 36.06 / 100 = 0.3606
Step 6: 0.3606 x 109 = 39.2854
Step 7: 39.2854 - 109 = -69.7146
Step 8: -69.7146 / 100 = -0.697146
Step 9: -0.697


In [ ]:
"Increase 109 by 25%.\nTo increase 109 by 25%, we need to follow these steps:\n\n1. Calculate 25% of 109.\n2. Add the result from step 1 to 109.\n\nFirst, let's calculate 25% of 109. We do this by multiplying 109 by 0.25 (since 25% is equivalent to 0.25 in decimal form).\n\n\\[ 109 \\times 0.25 = 27.25 \\]\n\nNext, we add this result to 109.\n\n\\[ 109 + 27.25 = 136.25 \\]\n\nTherefore, increasing 109 by 25% gives us \\(\\boxed{136.25}\\)."


In [59]:
response

"Increase 109 by 25%.\nTo increase 109 by 25%, we need to follow these steps:\n\n1. Calculate 25% of 109.\n2. Add the result from step 1 to 109.\n\nFirst, let's calculate 25% of 109. We do this by multiplying 109 by 0.25 (since 25% is equivalent to 0.25 in decimal form).\n\n\\[ 109 \\times 0.25 = 27.25 \\]\n\nNext, we add this result to 109.\n\n\\[ 109 + 27.25 = 136.25 \\]\n\nTherefore, increasing 109 by 25% gives us \\(\\boxed{136.25}\\)."

In [79]:
response

'1. Convert 25% to a decimal: 25% = 0.25\n2. Multiply 109 by 0.25: 109 * 0.25 = 27.25\n3. Add the result to the original number: 109 + 27.25 = 136.25\nFinal answer: 136.25\n\nYour solution is correct.'

In [80]:
extract_number(response)

0.25

In [ ]:
response = generate_answer(test_problem, prompt_template="cot")
print(response)
print(100*"-")
extracted = extract_number(response)
print(f"  Extracted: {extracted}\n")


1. Convert 25% to a decimal: 25% = 0.25
2. Multiply 109 by 0.25: 109 * 0.25 = 27.25
3. Add the result to the original number: 109 + 27.25 = 136.25
Final answer: 136.25

Your solution is correct.
----------------------------------------------------------------------------------------------------
  Extracted: 0.25



In [58]:
response = generate_answer(test_problem, prompt_template="cot")
print(response)
print(100*"-")
extracted = extract_number(response)
print(f"  Extracted: {extracted}\n")


Increase 109 by 25%.
To increase 109 by 25%, we need to follow these steps:

1. Calculate 25% of 109.
2. Add the result from step 1 to 109.

First, let's calculate 25% of 109. We do this by multiplying 109 by 0.25 (since 25% is equivalent to 0.25 in decimal form).

\[ 109 \times 0.25 = 27.25 \]

Next, we add this result to 109.

\[ 109 + 27.25 = 136.25 \]

Therefore, increasing 109 by 25% gives us \(\boxed{136.25}\).
----------------------------------------------------------------------------------------------------
  Extracted: 27.25



In [57]:
response = generate_answer(test_problem, prompt_template="few_shot")
print(response)
print(100*"-")
extracted = extract_number(response)
print(f"  Extracted: {extracted}\n")

Problem: Increase 109 by 25%
Answer: 136.25

Problem: What is 76 + 55?
Answer: 131.0

Problem: Sarah has $286. She spends $128. How much money does she have left?
Answer: 158.0

Problem: What is the circumference of a circle with radius 14? (use π ≈ 3.14)
Answer: 87.92

Problem: What is the volume of a cube with side length 3?
Answer: 27.0

Problem: Increase 109 by 25%
Answer: 136.25

Problem: What is 76 + 55?
Answer: 131.0

Problem: Sarah has $286. She spends $128. How much money does she have left?
Answer: 158.0

Problem: What is the circumference of a circle with radius 14? (use π ≈ 3.14)
Answer: 87.92

Problem: What is the volume of a cube with side length 3?
Answer: 27.0

Problem: Increase 109 by 25%
Answer: 136.25

Problem: What is 76 + 55?
Answer: 131.0

Problem: Sarah has $286. She spends $128. How much money does she have left?
Answer: 158.0

Problem: What is
----------------------------------------------------------------------------------------------------
  Extracted: 136.2

## Part 6b: New extract_number function

In [81]:
def extract_number(text):
    """
    Extract the most relevant (usually last) number from text.
    Handles patterns like:
    - "The answer is 42"
    - "Result = 42.5"
    - "15 + 27 = 42"
    - "Final answer: 136.25"
    - Negative numbers: "-15"
    Returns None if no number is found.
    """
    # 1️⃣ Cherche explicitement les cas de 'answer', 'result', 'equals', '='
    pattern_priority = re.findall(r'(?:answer|result|equals?|=)\s*:?\s*(-?\d+\.?\d*)', text, re.IGNORECASE)
    if pattern_priority:
        # on prend le dernier trouvé pour être sûr
        try:
            return float(pattern_priority[-1])
        except ValueError:
            pass

    # 2️⃣ Sinon, prend le dernier nombre présent dans le texte
    all_numbers = re.findall(r'-?\d+\.?\d*', text)
    if all_numbers:
        try:
            return float(all_numbers[-1])
        except ValueError:
            pass

    return None


In [83]:
test_strings = [
    "The answer is 42",
    "42",
    "15 + 27 = 42",
    "Calculating... the result is 42.5!",
    "No number here",
    "The value is -15",
    '1. Convert 25% to a decimal: 25% = 0.25\n2. Multiply 109 by 0.25: 109 * 0.25 = 27.25\n3. Add the result to the original number: 109 + 27.25 = 136.25\nFinal answer: 136.25\n\nYour solution is correct.'
]

print("Number extraction tests:")
for s in test_strings:
    result = extract_number(s)
    print(f"  '{s}' -> {result}")


Number extraction tests:
  'The answer is 42' -> 42.0
  '42' -> 42.0
  '15 + 27 = 42' -> 42.0
  'Calculating... the result is 42.5!' -> 42.5
  'No number here' -> None
  'The value is -15' -> -15.0
  '1. Convert 25% to a decimal: 25% = 0.25
2. Multiply 109 by 0.25: 109 * 0.25 = 27.25
3. Add the result to the original number: 109 + 27.25 = 136.25
Final answer: 136.25

Your solution is correct.' -> 136.25


In [84]:
response = generate_answer(test_problem, prompt_template="cot")
print(response)
print(100*"-")
extracted = extract_number(response)
print(f"  Extracted: {extracted}\n")


Step 1: Convert 25% to a decimal: 25% = 0.25
Step 2: Multiply 109 by 0.25: 109 * 0.25 = 27.25
Step 3: Add the result to the original number: 109 + 27.25 = 136.25
Final Answer: 136.25
----------------------------------------------------------------------------------------------------
  Extracted: 136.25



## Part 7: Evaluate on Validation Set

Test your best prompting strategy on a subset of training data.

In [ ]:
# TODO: Choose your best prompt template
best_template = "cot"  # Change based on your experiments

# Evaluate on a small validation set (last 50 training examples)
val_data = train_data.tail(50)

predictions = []
ground_truth = val_data['solution'].tolist()

print(f"Evaluating on {len(val_data)} validation problems...\n")

for idx, row in val_data.iterrows():
    problem = row['problem']
    solution = row['solution']

    response = generate_answer(problem, prompt_template=best_template)
    prediction = extract_number(response)

    # If no number extracted, use 0 (will be wrong)
    if prediction is None:
        prediction = 0.0

    predictions.append(prediction)

    if (len(predictions) % 10) == 0:
        print(f"Processed {len(predictions)}/{len(val_data)} problems...")

# Calculate accuracy
accuracy = check_accuracy(predictions, ground_truth)
print(f"\nValidation Accuracy: {accuracy:.2%}")
print(f"Need to achieve: 70% on test set")

Evaluating on 50 validation problems...

Processed 10/50 problems...
Processed 20/50 problems...
Processed 30/50 problems...
Processed 40/50 problems...
Processed 50/50 problems...

Validation Accuracy: 24.00%
Need to achieve: 70% on test set


## Part 8: Generate Test Predictions

Generate predictions for the test set and create submission file.

In [ ]:
# TODO: Generate predictions on test set
print(f"Generating predictions on {len(test_data)} test problems...\n")

test_predictions = []

for idx, row in test_data.iterrows():
    problem = row['problem']

    response = generate_answer(problem, prompt_template=best_template)
    prediction = extract_number(response)

    # If no number extracted, use 0
    if prediction is None:
        prediction = 0.0
        print(f"⚠️  Warning: No number extracted for problem {idx}: {problem[:50]}...")

    test_predictions.append(prediction)

    if (idx + 1) % 10 == 0:
        print(f"Processed {idx + 1}/{len(test_data)} problems...")

print("\nAll test predictions generated!")

## Part 9: Create Submission File

Save predictions in the required format for evaluation.

In [ ]:
# Create submission DataFrame
submission = pd.DataFrame({
    'id': test_data['id'],
    'solution': test_predictions
})

# Save to CSV
submission.to_csv('submission.csv', index=False)

print("Submission file created: submission.csv")
print("\nSubmission preview:")
print(submission.head(10))

# Verify all predictions are numerical
non_numeric = submission['solution'].isna().sum()
if non_numeric > 0:
    print(f"\n⚠️  WARNING: {non_numeric} predictions are not numerical!")
    print("These will result in incorrect answers. Please fix them.")
else:
    print("\n✓ All predictions are numerical")

# Show statistics
print("\nPrediction statistics:")
print(submission['solution'].describe())

## Part 10 (Optional): Fine-Tuning with LoRA

If prompting doesn't achieve 70% accuracy, consider fine-tuning with LoRA.

In [ ]:
# TODO: Implement LoRA fine-tuning (OPTIONAL)
from peft import LoraConfig, get_peft_model, TaskType
from torch.utils.data import Dataset, DataLoader

# This is a template - implement if needed
print("LoRA fine-tuning is optional.")
print("Use this if prompting strategies don't achieve 70% accuracy.")
print("\nConsider:")
print("- Prepare training dataset in correct format")
print("- Configure LoRA parameters (r=8, alpha=32)")
print("- Train for a few epochs")
print("- Evaluate and compare with prompting approaches")

## Questions

Answer the following questions:

1. **Which prompting strategy worked best and why?**
   - YOUR ANSWER HERE

2. **What types of math problems were most challenging for the model?**
   - YOUR ANSWER HERE

3. **How did you handle number extraction from model outputs?**
   - YOUR ANSWER HERE

4. **What are the limitations of using LLMs for mathematical reasoning?**
   - YOUR ANSWER HERE

5. **If you used LoRA fine-tuning, what were the trade-offs compared to prompting?**
   - YOUR ANSWER HERE (or N/A if you didn't use LoRA)